In [1]:
import yfinance as yf
import pandas as pd
import polars as pl
import os
import re
import urllib.parse
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math 
import pyarrow
import numpy as np
from io import StringIO
import time
import logging

Collect scores from hard memory


In [2]:
root = r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\Analysis\data\snp_500_scores'

ticker_symbol_pattern = re.compile(r'^([A-Z]+(?:\.[A-Z]+)?)\.parquet$')

df_dict_scores = {}

for dir in os.listdir(root):
    
    match = ticker_symbol_pattern.search(dir)

    ticker = match.group(1)
    ticker = ticker.replace('.', '-')
    print(ticker)

    file_path = os.path.join(root, dir)

    df = pl.read_parquet(file_path)
    df_dict_scores[ticker] = df 


A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FOXA
FRT
FSLR
FTNT
FTV
GD
GE
GEHC
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
IN

This is support func to find end prices and their dates. It takes as an arguments starting index, which is the nuber of row of hist dataframe and this hist dataframe itself. Hist dataframe contain daily OHLC data for particular ticker (company). 

It also cheks wether index of end price is in hist, meaning that if this particular report was released very resently (less then 8 trading days ago) it will append none values for end prices out of range of hist df instead of raising an error.

It returns lists of end prices and corresponding dates (none values for prices and their dates, if they are out of range of hist df).

In [3]:
def find_end_price(start_index, hist):
    end_price_list = []
    end_price_date_list = []
    
    for x in range(2, 8):
        idx = start_index + x
        if idx < len(hist):
            end_price_list.append(hist.iloc[idx]['Open'])
            end_price_date_list.append(hist.index[idx])
        else:
            end_price_list.append(None)
            end_price_date_list.append(None)
    return end_price_list, end_price_date_list

This is support func to calc the regular returns. It takes start price and end_price_list as an arguments. It checks whether value in end price list is none and if it is, it appends none in returns list, instead of raising an error. So we are shure that all end price lists are of the same length, but for those prices that are not found in historical data we have none values for returns.

In [4]:
def regular_returns(start_price, end_price_list):
    retuns = []
    for end_price in end_price_list:
        if end_price is None:
            retuns.append(None)
        else:
            ret = (end_price - start_price) / start_price * 100
            retuns.append(ret)
    return retuns

This is the support func that calc SnP500 returns. It takes_start date and end_price_date_list as and arguments and then calculate returns for this timeframes. This func also checks whether value of end_price_date_list is none (meaning that it is not found in historical data), and if it is, func will append none instead of raising an error.

Now we are shure that list of snp_returns will be of equal length for all report release dates and all time frames, but for those returns that cannot be calculates due to non existent end prices we will have nones 

In [5]:
def snp_500_return(start_date, end_price_date_list, snp_price):

    snp_returns = []

    for end_date in end_price_date_list:
        if end_date is None:
            snp_returns.append(None)

        else: 
            start_index = snp_price.index.get_loc(start_date)
            start_price = snp_price.iloc[start_index]['Open']
    
            end_index = snp_price.index.get_loc(end_date)
            end_price = snp_price.iloc[end_index]['Open']
    
            ret = (end_price - start_price) / start_price * 100
            
            snp_returns.append(ret)

    return snp_returns

This is suppurt function that retrive the end quarter price. It take as an argument hist - historical price dataframe, date_str - the date of release of this particular report, df - company dataframe, with cols named with dates of releases of corresponding reports.

It takes the release date for current report and checks whether it is the last date in time series or not. If it is (this mean that we are are realy close to the end of hist df) we consider the end of hist df as the end of quarter. if it is not, we take next date in company df (which is the date of releas of the next quarterly report) and see its open price, this is pricisely the end of the quarter.

In [6]:
def find_end_quarter(hist, date_str, start_date, df):

    col_names = df.columns
    current_date_index = col_names.index(date_str)

    if current_date_index < len(col_names) - 1:

        next_date = col_names[current_date_index + 1]

        next_date_time_stemp = pd.Timestamp(next_date, tz='America/New_York')

        while next_date_time_stemp not in hist.index:

            next_date_time_stemp += pd.Timedelta(days=1)
    
        end_quarter_index = hist.index.get_loc(next_date_time_stemp)
        end_quarter_price = hist.iloc[end_quarter_index]['Open']
        end_quarter_date = hist.index[end_quarter_index]

        start_quarter_index = hist.index.get_loc(start_date)

        quarter_length = end_quarter_index - start_quarter_index

    else:
        end_quarter_price = None
        end_quarter_date = None
        quarter_length = None
    

    return end_quarter_price, end_quarter_date, quarter_length

This is a func that takes as an input a key-value pair from df_dict_scores and calculate 2-days, 3-days, 4-days, 5-days, 6-days, 7-days and full quarter excess returns for it (S&P500 is the benchmark). It stores the result as a polars df with dates of starting of the period as col names and corresponding returns as col values (seven in each: starting from 2-days down to 7-days and full quarter) - 

In [7]:
def computing_returns(tic_sym, df):

    company_ticker = yf.Ticker(tic_sym)
    hist = company_ticker.history(period="max")

    snp500_ticker = yf.Ticker("^GSPC")
    snp_price = snp500_ticker.history(period="max", auto_adjust=True)
    
    returns = {}

    for date_str in df.columns:

        start_date = pd.Timestamp(date_str, tz='America/New_York')
        
        while start_date not in hist.index:

            start_date += pd.Timedelta(days=1)
        

        start_index = hist.index.get_loc(start_date)

        start_price = hist.iloc[start_index]['Open']

        end_price_list, end_price_date_list = find_end_price(start_index, hist)
        
        if None not in end_price_list:
            end_quarter_price, end_quarter_date, length_of_quarter = find_end_quarter(hist, date_str, start_date, df)
            
        else:
            end_quarter_price = None
            end_quarter_date = None

        
        end_price_list.append(end_quarter_price)
        end_price_date_list.append(end_quarter_date)
        
        reg_returns = regular_returns(start_price, end_price_list)
        snp_returns = snp_500_return(start_date, end_price_date_list, snp_price)
        
        excess_returns = [a - b if a is not None and b is not None else None for a, b in zip(reg_returns, snp_returns)]
        
        #normalization: divide returns for each time frame by number of trading days (2-days returns/2, 3-days returns/3, ful quarter returns/length_of_quarter)
        timeframe_length = [2, 3, 4, 5, 6, 7, length_of_quarter]

        normalized_excess_returns = [x / y if x is not None and y is not None else None for x, y in zip(excess_returns, timeframe_length)]

        returns[date_str] = normalized_excess_returns

    returns = pl.DataFrame(returns)
    
    return returns


In [14]:
#example ticker symbol and correponding df
tic_sym = "HPE" #A CCI ETR DXCM HPE
df = df_dict_scores[tic_sym]
df

2019-03-05,2019-05-31,2019-08-30,2019-12-13,2020-03-09,2020-06-02,2020-09-03,2020-12-10,2021-03-04,2021-06-03,2021-09-03,2021-12-10,2022-03-03,2022-06-03,2022-09-01,2022-12-08,2023-03-07,2023-06-02,2023-09-01,2023-12-22
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.387597,-0.417376,-0.427632,-0.363463,-0.465278,-0.408589,-0.452184,-0.333063,-0.357027,-0.34139,-0.306502,-0.295492,-0.335582,-0.374286,-0.371875,-0.304416,-0.332117,-0.35725,-0.327103,-0.298287
-200.0,-245.0,-260.0,-764.0,-268.0,-333.0,-383.0,-821.0,-221.0,-226.0,-198.0,-708.0,-199.0,-262.0,-238.0,-772.0,-182.0,-239.0,-210.0,-766.0
0.40155,0.360132,0.366893,0.423795,0.413051,0.378623,0.379004,0.423603,0.45568,0.430206,0.435728,0.436763,0.441262,0.363149,0.364825,0.424925,0.43361,0.392541,0.37557,0.430761
1036.0,1093.0,1137.0,4310.0,1209.0,1633.0,1704.0,5018.0,1388.0,1504.0,1583.0,4997.0,1371.0,1356.0,1367.0,4961.0,1254.0,1463.0,1399.0,5114.0
16839.0,19758.0,20127.0,58696.0,18250.0,27044.0,28467.0,67793.0,18849.0,21610.0,22538.0,64482.0,19292.0,22587.0,22519.0,65313.0,17698.0,22418.0,22339.0,66854.0


In [15]:
computing_returns(tic_sym, df)

2019-03-05,2019-05-31,2019-08-30,2019-12-13,2020-03-09,2020-06-02,2020-09-03,2020-12-10,2021-03-04,2021-06-03,2021-09-03,2021-12-10,2022-03-03,2022-06-03,2022-09-01,2022-12-08,2023-03-07,2023-06-02,2023-09-01,2023-12-22
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.215868,-0.294977,-0.654996,-1.339723,1.258128,0.908542,0.768452,-0.220542,-0.579661,0.29824,-1.81623,-2.514099,-0.948997,1.443248,-0.77768,-0.417365,0.579611,0.285614,1.69348,0.439753
-0.421958,0.282097,-0.026144,-1.461293,-1.170659,3.313013,1.31977,0.017227,0.25847,-0.524653,-2.17942,-1.58676,-0.298386,1.049947,-0.731808,-0.469232,0.103525,1.185193,1.287741,-0.13027
-0.280495,-0.491918,0.751079,-0.950981,-0.549898,2.924998,0.433058,0.226753,0.643268,-0.26352,-1.311861,-1.148647,0.174512,0.419874,-0.739989,-0.384313,-0.383425,1.512522,0.568322,-0.005669
-0.732565,-0.584003,0.820078,-1.029944,-2.052784,1.963139,0.250496,0.019465,0.4607,-0.489865,-1.166739,-0.608916,-0.289722,0.394607,-0.525793,-0.192045,-0.505271,1.188302,0.474711,-0.174458
-0.653369,-0.446312,1.15911,-1.086409,-1.333427,1.140587,0.243644,-0.023588,0.981191,-0.548628,-0.934957,-0.342057,0.080947,0.309546,-0.383831,-0.30561,-0.712994,0.875177,0.036586,-0.143235
-0.42643,-0.263729,1.244342,-0.745236,-2.074586,0.153495,0.017141,-0.241404,0.781953,-0.2296,-1.30215,-0.236618,0.428481,0.308749,-0.277532,0.010151,-0.657322,1.113584,0.156507,0.161406
-0.191277,-0.095368,0.166924,-0.38741,-0.302548,-0.234418,0.318376,0.302626,0.019968,-0.153562,-0.029115,0.276737,-0.100262,-0.050254,0.304881,-0.140292,-0.105265,0.16727,-0.056067,null


This func calculate suprise variable. It takes ticker, and corresponding df, uses yfinance to retrive surprise value (in persentage points) and stack it as a row, to the bottom of the df. In case of an error it returns none and false.

In [10]:
def calc_suprise(tic_sym, df, period=10):
    periods = [50, 60, 70, 100, 1000]

    initial_delay = 1
    max_delay = 32

    eps = None

    delay = initial_delay
    for period in periods:
        
        try:
            company_ticker = yf.Ticker(tic_sym)
            eps = company_ticker.get_earnings_dates(limit=period)
            break
            
        except Exception as e:
            logging.error(f"Error calculating surprise for {tic_sym}: {e}")
        
        time.sleep(delay)
        delay = min(delay * 2, max_delay)
        
    if eps is None:
        return None, False  
    
    else:
        available_indices = list(eps.index)

        eps_dict = {}

        for col in df.columns:

            date_of_rep = pd.Timestamp(col, tz='America/New_York')
            closest_index = min(available_indices, key=lambda x: abs(x - date_of_rep))

            eps_surprise = eps.loc[closest_index, 'Surprise(%)']

            eps_dict[col] = eps_surprise

        eps_df = pl.DataFrame(eps_dict)   
        stacked_df = df.vstack(eps_df) 

        return stacked_df, True

In [11]:
calc_suprise(tic_sym, df)

(shape: (6, 20)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ 2019-02-2 ┆ 2019-05-0 ┆ 2019-08-0 ┆ 2019-11-0 ┆ … ┆ 2023-02-2 ┆ 2023-05-0 ┆ 2023-08-0 ┆ 2023-11- │
 │ 6         ┆ 3         ┆ 6         ┆ 5         ┆   ┆ 4         ┆ 4         ┆ 3         ┆ 02       │
 │ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
 │ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ -0.255306 ┆ -0.16469  ┆ -0.1504   ┆ -0.226037 ┆ … ┆ -0.331853 ┆ -0.153611 ┆ -0.151989 ┆ -0.21845 │
 │           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 8        │
 │ -1263.0   ┆ -125.0    ┆ -188.0    ┆ -316.0    ┆ … ┆ -2242.0   ┆ -151.0    ┆ -214.0    ┆ -374.0   │
 │ 0.452186  ┆ 0.514083  ┆ 0.47309   ┆ 0.472521  ┆ … ┆ 0.455583  ┆


This is an alternative function to calculate eps_srprise. In case of calc_suprise() failed to retrive it, this function attempts to retrive at least some values of srprise. In case of fail of this function it just fill null values instead.

In [12]:
def calc_suprise_sec_attempt(tic_sym, df):
    eps_dict = {}

    try:
        company_ticker = yf.Ticker(tic_sym)
        eps = company_ticker.earnings_dates

        available_indices = list(eps.index)

        eps_dict = {}

        retrived_dates = 0

        for col in sorted(df.columns, reverse=True):

            if available_indices:
                date_of_rep = pd.Timestamp(col, tz='America/New_York')

                closest_index = min(available_indices, key=lambda x: abs(x - date_of_rep))

                available_indices = [dt for dt in available_indices if dt < closest_index]

                eps_surprise = eps.loc[closest_index, 'Surprise(%)']

                eps_dict[col] = eps_surprise

                retrived_dates += 1
            else:
                eps_dict[col] = None

        failed_dates = len(df.columns) - retrived_dates

        eps_dict = {key: eps_dict[key] for key in sorted(eps_dict)}

    except Exception as e:
        logging.error(f"Error for second attempt surprise for {tic_sym}: {e}")

        for col in df.columns:
            eps_dict[col] = None

        failed_dates = len(df.columns)

    eps_df = pl.DataFrame(eps_dict)   
    stacked_df = df.vstack(eps_df)

    return stacked_df, failed_dates

calc_suprise_sec_attempt(tic_sym, df)

(shape: (6, 20)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ 2019-02-2 ┆ 2019-05-0 ┆ 2019-08-0 ┆ 2019-11-0 ┆ … ┆ 2023-02-2 ┆ 2023-05-0 ┆ 2023-08-0 ┆ 2023-11- │
 │ 6         ┆ 3         ┆ 6         ┆ 5         ┆   ┆ 4         ┆ 4         ┆ 3         ┆ 02       │
 │ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
 │ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ -0.255306 ┆ -0.16469  ┆ -0.1504   ┆ -0.226037 ┆ … ┆ -0.331853 ┆ -0.153611 ┆ -0.151989 ┆ -0.21845 │
 │           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 8        │
 │ -1263.0   ┆ -125.0    ┆ -188.0    ┆ -316.0    ┆ … ┆ -2242.0   ┆ -151.0    ┆ -214.0    ┆ -374.0   │
 │ 0.452186  ┆ 0.514083  ┆ 0.47309   ┆ 0.472521  ┆ … ┆ 0.455583  ┆

This is the general loop, that takes each key-value pair from df_dict_scores, applies computing_returns() to it, then verticaly stackes the result with orig df and stores it in the new dict

In [16]:
dict_for_anal = {}
row_names = pl.Series("row_names", ["lm_polarity", "lm_raw", "hiv4_polarity", "hiv4_raw", "doc_length", "eps_surprise", "2_day_reterns","3_day_reterns", "4_day_reterns", "5_day_reterns", "6_day_reterns", "7_day_reterns", "full_quarter_returns"])

company_fails = 0
doc_fails = 0
total_docs = 0

for tic_sym, df in df_dict_scores.items():
    total_docs += df.shape[1]
    print(tic_sym)
    
    regressors_df, sucssess = calc_suprise(tic_sym, df)

    if sucssess == False:
        company_fails +=1
        regressors_df, failed_dates = calc_suprise_sec_attempt(tic_sym, df)
        doc_fails += failed_dates

    returns_df = computing_returns(tic_sym, df)
    
    stacked_df = regressors_df.vstack(returns_df)
    stacked_df_with_indx = stacked_df.hstack([row_names])
    
    dict_for_anal[tic_sym] = stacked_df_with_indx

print(f'Company fails ratio: {company_fails/497}')
print(f'Documents fails ratio: {doc_fails/total_docs}')


A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA


ERROR:root:Error calculating surprise for DVA: 'Earnings Date'
ERROR:root:Error calculating surprise for DVA: 'Earnings Date'
ERROR:root:Error calculating surprise for DVA: 'Earnings Date'
ERROR:root:Error calculating surprise for DVA: 'Earnings Date'
ERROR:root:Error calculating surprise for DVA: 'Earnings Date'
ERROR:root:Error for second attempt surprise for DVA: 'Earnings Date'


DVN


ERROR:root:Error calculating surprise for DVN: 'Earnings Date'
ERROR:root:Error calculating surprise for DVN: 'Earnings Date'
ERROR:root:Error calculating surprise for DVN: 'Earnings Date'
ERROR:root:Error calculating surprise for DVN: 'Earnings Date'
ERROR:root:Error calculating surprise for DVN: 'Earnings Date'
ERROR:root:Error for second attempt surprise for DVN: 'Earnings Date'


DXCM


...

T
TAP
TDG
TDY
TECH
TEL
TER
TFC
TFX
TGT
TJX
TMO
TMUS
TPR
TRGP
TRMB
TROW
TRV
TSCO
TSLA
TSN
TT
TTWO
TXN
TXT
TYL
UAL
UBER
UDR
UHS
ULTA
UNH
UNP
UPS
URI
USB
V
VICI
VLO
VLTO
VMC
VRSK
VRSN
VRTX
VST
VTR
VTRS
VZ


ERROR:root:Error calculating surprise for VZ: 'Earnings Date'
ERROR:root:Error calculating surprise for VZ: 'Earnings Date'
ERROR:root:Error calculating surprise for VZ: 'Earnings Date'
ERROR:root:Error calculating surprise for VZ: 'Earnings Date'
ERROR:root:Error calculating surprise for VZ: 'Earnings Date'
ERROR:root:Error for second attempt surprise for VZ: 'Earnings Date'


WAB


ERROR:root:Error calculating surprise for WAB: 'Earnings Date'
ERROR:root:Error calculating surprise for WAB: 'Earnings Date'
ERROR:root:Error calculating surprise for WAB: 'Earnings Date'
ERROR:root:Error calculating surprise for WAB: 'Earnings Date'
ERROR:root:Error calculating surprise for WAB: 'Earnings Date'
ERROR:root:Error for second attempt surprise for WAB: 'Earnings Date'


WAT
WBA
WBD
WDC
WEC
WELL
WFC
WM
WMB
WMT
WRB
WRK
WST
WTW
WY
WYNN
XEL
XOM
XYL
YUM
ZBH
ZBRA
ZTS
Company fails ratio: 0.028169014084507043
Documents fails ratio: 0.026856561546286878


In [17]:
dict_for_anal["AEE"]

2019-02-26,2019-05-10,2019-08-02,2019-11-08,2020-02-28,2020-05-11,2020-08-07,2020-11-05,2021-02-22,2021-05-10,2021-08-06,2021-11-04,2022-02-23,2022-05-06,2022-08-08,2022-11-04,2023-02-22,2023-05-05,2023-08-03,2023-11-09,row_names
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
-0.211316,-0.202797,-0.192547,-0.19604,-0.242243,-0.234177,-0.320207,-0.306324,-0.266361,-0.23224,-0.246114,-0.213582,-0.238938,-0.242574,-0.227633,-0.214206,-0.224048,-0.192982,-0.2,-0.167592,"""lm_polarity"""
-366.0,-87.0,-93.0,-99.0,-406.0,-148.0,-309.0,-310.0,-639.0,-170.0,-190.0,-195.0,-594.0,-196.0,-201.0,-193.0,-559.0,-154.0,-167.0,-151.0,"""lm_raw"""
0.470266,0.508287,0.502341,0.520712,0.479986,0.50509,0.433089,0.439725,0.460614,0.464941,0.453013,0.46332,0.476764,0.477736,0.452534,0.44806,0.468905,0.456778,0.437378,0.442765,"""hiv4_polarity"""
4286.0,1288.0,1395.0,1521.0,4065.0,1439.0,1890.0,2050.0,5368.0,1724.0,1827.0,2160.0,5899.0,1899.0,2045.0,2148.0,6062.0,1934.0,2015.0,2139.0,"""hiv4_raw"""
58108.0,17100.0,19018.0,20193.0,54567.0,19648.0,31067.0,33618.0,75857.0,26740.0,29809.0,34186.0,81542.0,28414.0,33272.0,35349.0,85344.0,29831.0,33058.0,34479.0,"""doc_length"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-0.084395,0.761773,-0.113404,0.415435,0.362242,-0.028741,0.261048,-0.942138,0.005085,0.143593,0.52145,-0.211858,0.070615,1.608915,0.240994,0.149262,-0.634929,-0.271307,-0.990158,-1.021739,"""4_day_reterns"""
0.061359,0.695961,0.023147,0.469176,0.675814,-0.003203,0.400904,-0.897998,-0.17366,-0.054814,0.286064,0.036607,-0.003176,0.899509,0.042316,-0.02831,-0.74731,-0.359163,-0.687646,-0.514446,"""5_day_reterns"""
0.064108,0.911952,0.27154,0.381921,0.70249,0.115397,0.224893,-0.848674,-0.220383,-0.145378,0.414663,0.079055,-0.052909,0.601667,0.122527,-0.498794,-0.70567,-0.52526,-0.570334,-0.699415,"""6_day_reterns"""


This func takes as input key-value pair (company name and dataframe with sentiment scores and returns), regress each retern (2_days, 3_days, etc) on polarity score and raw sentiment score. Then it collects the slopes (beta-coefficient) of returns regressed on each score in the separate polars series (in the loop they will be staked in dataframe)

In [18]:
def add_quarter_dummy(company_df_pd, var_names):
    
    dates = pd.to_datetime(company_df_pd.columns)
   
    date_series = pd.Series(dates.quarter, index=dates)
   
    dummies = pd.get_dummies(date_series, prefix="Q")
    dummies.index = dummies.index.strftime('%Y-%m-%d')
    
    all_quarters = ["Q_1", "Q_2", "Q_3", "Q_4"]
    dummies = dummies.reindex(columns=all_quarters, fill_value=False)
   
    dummies = dummies.drop(columns=["Q_4"])
    
    dummies = dummies.transpose()
    df_wit_dum = pd.concat([company_df_pd, dummies])
    
    df_wit_dum = df_wit_dum.transpose()
   
    for col in df_wit_dum.columns:
        df_wit_dum[col] = pd.to_numeric(df_wit_dum[col])

    boolean_cols = ['Q_1', 'Q_2', 'Q_3']
    df_wit_dum[boolean_cols] = df_wit_dum[boolean_cols].astype(int)

    res_col_names = var_names + boolean_cols

    df_wit_dum.columns = res_col_names

    return df_wit_dum

    

In [19]:
def fit_regression(y, x):

    x = sm.add_constant(x)
    model = sm.OLS(y, x, missing='drop').fit()
    
    slope = model.params.iloc[1]
    
    return slope

In [25]:
def regression_slopes(df_wit_dum, X_names, company_name):
    
    for col in df_wit_dum.select_dtypes(include=['bool']).columns:
        df_wit_dum[col] = df_wit_dum[col].astype(int)
        
    df_wit_dum = df_wit_dum.apply(pd.to_numeric, errors='coerce')

    # x in range(0, 4) represents set of independent variables; 4th column represents control variable for document length; -3, -2, -1 columns represent dummy variables for quarters
    
    X = [df_wit_dum.iloc[:, [x, 4, 5, -3, -2, -1]] for x in range(0, 4)]

    Y = [df_wit_dum.iloc[:, [y]] for y in range(6, 13)]

    slopes_dict = {}

    for i, x in enumerate(X):

        name_of_score = X_names[i][0].split()[0]
        slopes_name = f'{name_of_score}_slopes'
        
        slopes = []
        for y in Y:
    
            slope = fit_regression(y, x)
            slopes.append(slope)

        slopes_dict[slopes_name] = pl.Series(company_name, slopes)
    
    return slopes_dict

This is the func checks if there is enough data in single company dataframe. As an argument it takes a company df with out all dummies and check that each col contaian enough data to fit the model. 

In [24]:
def check_6_nonnull(df):
    df = df.transpose()

    counts = df.count()
    mask = counts < 6
    if mask.sum_horizontal().item() > 0:
        return False
    else:
        return True

This is the general loop that takes dict_for_anal whith scores and returns and for each company in the dict it calculates dict with four dfs: first one contain regression slopes for lm_polarity scores, second one for lm_raw scores and so on for each regressor. Serieses stacked in the separete dataframes (for raw and for polarity scores) 

Note: sector dummies are not included because each regression estimated for a single company (and it obviosly has the same sector variable value)

In [28]:
def compute_all_slopes(dict_for_anal):

    slopes_df_dict = {
        "lm_polarity_slopes_df" : pl.DataFrame(),
        "lm_raw_slopes_df" : pl.DataFrame(),
        "hiv4_polarity_slopes_df" : pl.DataFrame(),
        "hiv4_raw_slopes_df" : pl.DataFrame()
    }

    fails = 0

    for company_name, company_df in dict_for_anal.items(): 

        X_names = [company_df.select(pl.col("row_names")).row(x) for x in range(0, 4)]

        var_names = company_df.get_column("row_names").to_list()
        company_df_woi = company_df.select(pl.exclude("row_names"))

        company_df_woi = company_df_woi.to_pandas()
        print(company_name)

        df_wit_dum = add_quarter_dummy(company_df_woi, var_names)

        enough_data = check_6_nonnull(company_df)

        if enough_data == True:

            slopes_dict = regression_slopes(df_wit_dum, X_names, company_name)
    
            for x in X_names:
                
                name_of_score = x[0].split()[0]
                slopes_name = f'{name_of_score}_slopes'
                
                df_name = f"{name_of_score}_slopes_df"
                
                slopes_df_dict[df_name] = slopes_df_dict[df_name].hstack([slopes_dict[slopes_name]])
        else:
            fails +=1
    print(f'fails rate: {fails/len(dict_for_anal)}')

    return slopes_df_dict
        

In [29]:
slopes_df_dict = compute_all_slopes(dict_for_anal)

A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FOXA
FRT
FSLR
FTNT
FTV
GD
GE
GEHC
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
IN

As an output from previous functions we have dataframes with regression slopes of different returns on raw, polarity scores and Loughran-McDonald and Harvard dictionaries. Each col of these dfs contain regression slopes for particular company. First row contain slope for 2_days return, second row contain slope for 3_days return, etc.  I have decided to check what is the proportion of positive regression slopes for each return timeframe (2_days, 3_days, etc). For that purpose i will transpose each of these dfs, apply condition > 0, then verticaly sum boolean values and obtain the prorortion of positive slopes for each time frame. 

In [30]:
def calc_efficiency_metrics(slopes_df):

    df_tranposed = slopes_df.transpose()

    res = df_tranposed.select(pl.all() > 0).sum()
    total_num_of_slopes = slopes_df.shape[1]

    positive_slopes = [res.select(pl.col(res.columns[x]).gather(0)).item() for x in range(slopes_df.shape[0])]

    for counter, x in enumerate(positive_slopes):
        print(f'Prorortion of positive {counter + 2}_day returns: {x/total_num_of_slopes}')

This loop apply calc_efficiency_metrics function to each item of slopes_df_dict where each item is a dataframe which regression slopes for each regressor.

Note: actually there is no 8_day returns, it is actually full-quarter returns (just easier to print it programmatically)

In [32]:
for key, slopes_df in slopes_df_dict.items():
    print("\n")
    print(f'Metrics for: {key}')
    calc_efficiency_metrics(slopes_df)



Metrics for: lm_polarity_slopes_df
Prorortion of positive 2_day returns: 0.4760914760914761
Prorortion of positive 3_day returns: 0.4885654885654886
Prorortion of positive 4_day returns: 0.49480249480249483
Prorortion of positive 5_day returns: 0.4885654885654886
Prorortion of positive 6_day returns: 0.4864864864864865
Prorortion of positive 7_day returns: 0.47193347193347196
Prorortion of positive 8_day returns: 0.46153846153846156


Metrics for: lm_raw_slopes_df
Prorortion of positive 2_day returns: 0.4698544698544699
Prorortion of positive 3_day returns: 0.4594594594594595
Prorortion of positive 4_day returns: 0.4656964656964657
Prorortion of positive 5_day returns: 0.45114345114345117
Prorortion of positive 6_day returns: 0.47193347193347196
Prorortion of positive 7_day returns: 0.4760914760914761
Prorortion of positive 8_day returns: 0.4553014553014553


Metrics for: hiv4_polarity_slopes_df
Prorortion of positive 2_day returns: 0.46361746361746364
Prorortion of positive 3_day re

Positive slopes means that sentiment scores and returns mooves in the same direction (hense, they are working as predictors of stock price movement). As the result of this preliminary analysis we can say that sentiment scores of quarterly reports are not very good predictors of stock movments. Positive proportion of this slopes are close to 0.5, which means that on average in our sample there are 50/50 chance that scores and returns are mooving in the same direction.

This is the separate function for regression fitting, that will be used to fit regression for all companies together. Returns has no autocorrelation, but very likely exhibit heteroscedasticity. To handle this problem White's SE are used to obtain pobust estimation of p_val

In [33]:
def fit_rob_se(y, x):

    x = sm.add_constant(x)
    model = sm.OLS(y, x, missing='drop').fit()
    robust_se = model.get_robustcov_results(cov_type='HC0')
    
    return robust_se.summary()

This function is used to get parametrs of the model and conviniently display them. As an argument it takes regression model summary and returns a table only wit significant coefficients. Since we have 4 different sets of regressors and 7 dependent variables, in total we have 28 regressions. So, with help of this function, only those parametrs with p_val < 5% will be displaied as an output of regression model fit. 

In [34]:
def extract_params(summary):
    
    coef_table = summary.tables[1].as_html()
    coef_table_io = StringIO(coef_table)
    df = pd.read_html(coef_table_io, header=0, index_col=0)[0]

    df.columns = ['coef', 'std err', 't', 'P>|t|', '[0.025', '0.975]']
    
    df = df.reset_index()
    df = df.rename(columns={'index': 'Variable'})

    filtered_df = df[df['P>|t|'] < 0.05]
    
    return filtered_df


This is function is used to regress dependent variables on all set of regressors using all dummies (sector dummies included). It takes total df and variadle manes as an input, and prints significant coefficients as an output for each regression.

In [57]:
def regression_single_slope(df_wit_dum, X_names, Y_names):

    for col in df_wit_dum.select_dtypes(include=['bool']).columns:
        df_wit_dum[col] = df_wit_dum[col].astype(int)

    df_wit_dum = df_wit_dum.apply(pd.to_numeric, errors='coerce')

    # x in range(0, 4) represents set of independent variables; 4th column represents control variable for document length; 5th column represent control variable for eps surprize in persentage points; from -14 to -1 columns represent dummy variables for quarters and economic sector.
    X = [df_wit_dum.iloc[:, [x, 4, 5, -14, -13, -12, -11, -10, -9,-8 ,-7, -6, -5 ,-4, -3, -2, -1]] for x in range(0, 4)]

    Y = [df_wit_dum.iloc[:, [y]] for y in range(6, 13)]

    for i, x in enumerate(X):

        name_of_score = X_names[i][0]

        for j, y in enumerate(Y):
            returns_timeframe = Y_names[j][0]
            
            summary = fit_rob_se(y, x)
            params_df = extract_params(summary)
            if not params_df.empty:
                print(f' X variable: {name_of_score}', '\n', f'Y variable: {returns_timeframe}', '\n')
                print(f'Significant coefficients: \n {params_df}', '\n\n\n') 

This function adds a sector value to data. As an input it takes a dataframe and company name. It uses yfinance library to extract sector value and add it as a column of categorical values to a df.

In [37]:
def addsector_value(company_name, df_wit_dum):
    
    ticker = yf.Ticker(company_name)
    company_info = ticker.info

    sector = company_info.get('sector', 'Sector information not available')
    df_wit_dum['sector'] = sector

    return df_wit_dum
    

This func stackes df of different companies in one df, apply get_dummies metod, to transform categorical data to actuall dummy variable set and then stores the result in the corresponding variables.

In [41]:
def compute_all_var_set(dict_for_anal):

    df_list = []
    
    for company_name in dict_for_anal:


        X_names = [dict_for_anal[company_name].select(pl.col("row_names")).row(x) for x in range(0, 4)]
        Y_names = [dict_for_anal[company_name].select(pl.col("row_names")).row(x) for x in range(6, 13)] 

        var_names = dict_for_anal[company_name].get_column("row_names").to_list()

        company_df_woi = dict_for_anal[company_name].select(pl.exclude("row_names"))
        company_df_pd = company_df_woi.to_pandas(use_pyarrow_extension_array=True)

        df_wit_dum = add_quarter_dummy(company_df_pd, var_names)
        df_wit_sector = addsector_value(company_name, df_wit_dum)

        df_list.append(df_wit_sector)
        
    total_df = pd.concat(df_list, ignore_index=True)
    total_df_with_dummies = pd.get_dummies(total_df, columns=['sector'])
    
    return total_df_with_dummies, X_names, Y_names


In [44]:
total_df_with_dummies, X_names, Y_names = compute_all_var_set(dict_for_anal)

This is the result of fitting regression on total data set. Almost in all cases only covariate variables has significant coeffisients. There are to exceptions: 
1. hiv4_polarity score for full_quarter_returns has coefficient -0.0751 with P>|t| 0.045.
    Negative sign suggests that this score moves in the opposit derection with full_quarter_returns (the opposit is expected).
    P_val is barely less 5% level (4.5%). Despite it's significants this is more likely to be a type one error.
2. hiv4_raw score for full_quarter_returns has coefficient -0.000020 with P>|t| 0.015.
    Slope has tiny ablosute value, so it is economicly insignificant, and would be regected at 1% significance level.

In [58]:
regression_single_slope(total_df_with_dummies, X_names, Y_names)

 X variable: lm_polarity 
 Y variable: 2_day_reterns 

Significant coefficients: 
             Variable      coef   std err      t  P>|t|        [0.025    0.975]
2         doc_length  0.000003  0.000001  2.344  0.019  4.930000e-07  0.000006
4                Q_1 -0.147500  0.060000 -2.439  0.015 -2.660000e-01 -0.029000
5                Q_2 -0.189000  0.055000 -3.410  0.001 -2.980000e-01 -0.080000
6                Q_3 -0.131600  0.055000 -2.408  0.016 -2.390000e-01 -0.024000
17  sector_Utilities -0.184800  0.060000 -3.067  0.002 -3.030000e-01 -0.067000 



 X variable: lm_polarity 
 Y variable: 3_day_reterns 

Significant coefficients: 
             Variable      coef       std err      t  P>|t|        [0.025  \
2         doc_length  0.000002  9.950000e-07  2.013  0.044  5.220000e-08   
4                Q_1 -0.091300  4.600000e-02 -1.981  0.048 -1.820000e-01   
5                Q_2 -0.150700  4.200000e-02 -3.575  0.000 -2.330000e-01   
6                Q_3 -0.082300  4.100000e-02 -2.022 

jupyter nbconvert --to html Parsim-sec\src\Analysis\excess_ret_500.ipynb